In [1]:
from datetime import date
from pathlib import Path

from loguru import logger
import sentier_data_tools as sdt

import os 

import brightway2 as bw
import lca_algebraic as agb
import numpy as np
from sympy import init_printing
import bw2io
import bw2data as bd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', 500)
from sentier_data_tools import (
    Demand,
    Flow,
    FlowIRI,
    GeonamesIRI,
    ModelTermIRI,
    DatasetKind,
    ProductIRI,
    SentierModel,
)
# Pretty print for Sympy
init_printing()

In [140]:
bw.projects.set_current('PCB_Project')

bw.databases

Initial setup already done, skipping


Databases dictionary with 3 object(s):
	MyForeground
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [141]:
EcoInvent = "ecoinvent-3.10-cutoff"
USER_DB = 'MyForeground'
#agb.setForeground(DB_Buck)
# This is better to cleanup the whole foreground model each time, and redefine it in the notebook (or a python file)
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)
# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

[WARNING] Db MyForeground was here. Reseting it


,backend,nb_activities,type
name,,,
ecoinvent-3.10-biosphere,sqlite,4363,biosphere
ecoinvent-3.10-cutoff,sqlite,23523,background
MyForeground,sqlite,0,foreground


In [10]:
class PCB_Model(SentierModel):
    provides = {
        ProductIRI(
            "link"
        ): "Printed Wiring Board",
    }
    aliases = {
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/layers_number"
        ): "layers_number",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/base_material"
        ): "base_material",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/percent_fiber_glass"
        ): "percent_fiber_glass",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/board_thickness"
        ): "board_thickness",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/copper_thickness"
        ): "copper_thickness",
        ModelTermIRI(
            "https://vocab.sentier.dev/model-terms/pcb/mounting_type"
        ): "mounting_type",
        
        ProductIRI("https://vocab.sentier.dev/products/pcb"): "Electronic_factory",
    }

    def get_electrolysis_inventory(self) -> None:
        bom_electrolysis = self.get_model_data(
            self, product=self.hydrogen, kind=DatasetKind.BOM
        )

    def run(self) -> tuple[list[Demand], list[Flow]]:
        self.prepare()

In [15]:
def create_pcb_local_datastorage(reset: bool = True):
    if reset:
        sdt.reset_local_database()

    df = pd.read_excel('Inventory_formated.xlsx')
    assert len(COLUMNS) == len(UNITS)
    assert len(COLUMNS) == len(df.columns) 

    metadata = sdt.Datapackage(
        name="pcb production data from EcoInvent3.10",
        description="inventory taken in EcoInvent3.10 database",
        contributors=[
            {
                "title": "Yannis Rosset",
                "path": "I don't have any website !",
                "role": "author",
            },
        ],
        homepage="https://ecoquery.ecoinvent.org/3.10/cutoff/dataset/9764/documentation",
    ).metadata()
    metadata.pop("version")

    for kind, iri in TYPES:
        copy = df.copy()
        logger.info("Adding {} records for type {}", len(copy), kind)
#        copy.drop(labels=["Electrolysis type"], axis="columns", inplace=True)
        copy.columns = COLUMNS

        sdt.Dataset(
            name=f"pcb process model",
            dataframe=copy,
            product=iri,
            columns=[{"iri": x, "unit": y} for x, y in zip(COLUMNS, UNITS)],
            metadata=metadata,
            location="https://www.geonames.org/6295630/",
            version=1,
            valid_from=date(2018, 1, 1),
            valid_to=date(2028, 1, 1),
        ).save()


COLUMNS = [
    "https://vocab.sentier.dev/model-terms/generic/product",  #pcb
    "https://vocab.sentier.dev/model-terms/energy/electricity", #electricity
    "https://vocab.sentier.dev/model-terms/energy/heat",  #heat
    "https://vocab.sentier.dev/flows/envo/ENVO_00003065",  #ultrapure water
    "https://vocab.sentier.dev/model-terms/electronic_factory", 
    "http://data.europa.eu/xsp/cn2024/740300000080", #copper
    "http://data.europa.eu/xsp/cn2024/701913000080", #glass fiber
    "http://data.europa.eu/xsp/cn2024/390730000080", # Epoxy
    "http://data.europa.eu/xsp/cn2024/280600000080", #HCl
    "http://data.europa.eu/xsp/cn2024/284700000080", #H2O2
    "http://data.europa.eu/xsp/cn2024/282739200080", #Fe (III) Cl
    "http://data.europa.eu/xsp/cn2024/710800000080", #Au
    "http://data.europa.eu/xsp/cn2024/750610000080", #Ni
    "http://data.europa.eu/xsp/cn2024/710691000080", #Ag
    "http://data.europa.eu/xsp/cn2024/250100000080", #NaCl
    "http://data.europa.eu/xsp/cn2024/281512000080", #NaOH
    "http://data.europa.eu/xsp/cn2024/280700000080", #Sulfuric acid
    "http://data.europa.eu/xsp/cn2024/290919900080",  #Solvent
    "https://vocab.sentier.dev/flows/envo/ENVO_00003096",  # water
    "https://vocab.sentier.dev/model-terms/pcb/pad_number", #Pad number
    "https://vocab.sentier.dev/model-terms/pcb/pad_surface", #surface area per pad

]

UNITS = [
    "https://www.w3.org/2001/XMLSchema#string",
    "https://vocab.sentier.dev/units/unit/KiloW-HR",
    "https://vocab.sentier.dev/units/unit/KiloW-HR",
    "https://vocab.sentier.dev/units/unit/M3",
    "https://vocab.sentier.dev/units/unit/NUM", #part of a factory
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM", #epoxy
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/KiloGM",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/L",
    "https://vocab.sentier.dev/units/unit/NUM",
    "https://vocab.sentier.dev/units/unit/MilliM2",
   
]

TYPES = [
    ("PCB", "https://vocab.sentier.dev/products/pcb"),
]



In [16]:
sdt.__version__

'0.5'

In [17]:
create_pcb_local_datastorage(True)

2024-10-10 15:53:02.272 | INFO     | __main__:create_pcb_local_datastorage:25 - Adding 20 records for type PCB


In [20]:
df_treatment = pd.read_excel('Inventory_formated.xlsx')
df_treatment.corr()

,Layer number,Electricity,Energy,ultrapure water,Electronic component factory,CU input,GF input,Epoxy input,HCL,H2O2,...,Au,Ni,Ag,NaCl,NaOH,Sulfuric Acid,Solvent,Water,Pad number,Surface area per pad
Layer number,1,1,1,NaN,NaN,0.75978,NaN,-1,0.849747,0.909577,...,-0.225494,0.945761,1,1,0.903421,0.964613,0.389157,0.946524,NaN,NaN
Electricity,1,1,1,NaN,NaN,1,NaN,NaN,1,1,...,NaN,1,NaN,NaN,1,1,NaN,1,NaN,NaN
Energy,1,1,1,NaN,NaN,1,NaN,NaN,1,1,...,NaN,1,NaN,NaN,1,1,NaN,1,NaN,NaN
ultrapure water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Electronic component factory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CU input,0.75978,1,1,NaN,NaN,1,NaN,1,0.817335,0.883616,...,0.982312,0.9632,1,1,0.9403,0.978427,0.762879,0.963831,NaN,NaN
GF input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Epoxy input,-1,NaN,NaN,NaN,NaN,1,NaN,1,NaN,NaN,...,NaN,NaN,NaN,NaN,-1,NaN,-1,NaN,NaN,NaN
HCL,0.849747,1,1,NaN,NaN,0.817335,NaN,NaN,1,0.991977,...,1,0.632394,1,1,0.753658,0.680672,1,0.634218,NaN,NaN
H2O2,0.909577,1,1,NaN,NaN,0.883616,NaN,NaN,0.991977,1,...,1,0.725251,1,1,0.830703,0.767825,1,0.726872,NaN,NaN
